In [1]:
using ITensors, ITensorMPS
using LinearAlgebra
# using SparseArrays
# using Plots
# using JLD2

In [2]:
include("../src/main.jl")
include("../src/observables.jl")
include("../src/states.jl")
include("../src/doubled_measurements.jl")

forced_doubled_measure_with_val (generic function with 2 methods)

In [3]:
function MPS_update_data(ρ::MPS, observables::Vector{Symbol}, data::Dict{Symbol,Vector{Float64}}, t::Int; ref=false)
    if :Ic in observables
        data[:Ic][t] = Ic2(ρ)
    end
    if :SR in observables
        data[:SR][t] = SR(ρ)
    end
    if :κEA in observables
        data[:κEA][t] = κEA(ρ; ref=ref)
    end
    if :κ2 in observables
        data[:κ2][t] = κ2(ρ; ref=ref)
    end
    if :maxlinkdim in observables
        data[:maxlinkdim][t] = maxlinkdim(ρ)
    end
    return data
end

MPS_update_data (generic function with 1 method)

In [4]:
function circuit(L::Int, T::Int, λ::Float64, δ::Float64, q::Float64; observables=Symbol[], cutoff=1E-8, maxdim=200)
    ρ, sites = ghz(L; ref=true)

    data = Dict([s => zeros(Float64, 2T+2) for s in observables])
    data = MPS_update_data(ρ, observables, data, 1; ref=true)
    data = MPS_update_data(ρ, observables, data, 2; ref=true)

    λzz = δ*(1-λ)
    λx = δ*λ

    Xn = decoherence_layer(sites, PauliX, q, 1:L, L)
    ZZn = decoherence_layer(sites, kron(PauliZ, PauliZ), q, 1:L-1, L)
    bell_state = bell(sites)


    for t in 1:T
        ρ, _, _ = doubled_measure(ρ, PauliX, λx, 1:L; ref=true)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        ρ = apply(Xn, ρ)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        data = MPS_update_data(ρ, observables, data, 2t+1; ref=true)

        ρ, _, _ = doubled_measure(ρ, kron(PauliZ,PauliZ), λzz, 1:L-1; ref=true)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        ρ = apply(ZZn, ρ)
        ρ /= inner(bell_state, ρ)
        truncate!(ρ; cutoff=cutoff, maxdim=maxdim)

        data = MPS_update_data(ρ, observables, data, 2t+2; ref=true)
    end

    return ρ, data
end

circuit (generic function with 1 method)

In [5]:
function sample(L::Int, T::Int, Δ::Float64, δ::Float64, q::Float64, samples::Int; observables=Symbol[], cutoff=1E-8, maxdim=200)
    mean_data = Dict([s => zeros(Float64, 2T+2) for s in observables])
    var_data = Dict([s => zeros(Float64, 2T+2) for s in observables])
    for _ in 1:samples
        _, sample_data = circuit(L, T, Δ, δ, q; observables=observables, cutoff=cutoff, maxdim=maxdim)
        for observable in observables
            mean_data[observable] += sample_data[observable]
            var_data[observable] += sample_data[observable].^2
        end
    end
    for observable in observables
        mean_data[observable] /= samples
        var_data[observable] /= samples
    end
    return mean_data, var_data
end

sample (generic function with 1 method)

In [7]:
circuit(2, 2, 0.1, 0.0, 0.4; observables=[:Ic, :SR, :κEA, :κ2, :maxlinkdim])

(MPS
[1] ((dim=2|id=480|"Qubit,Site,n=1"), (dim=2|id=729|"Link,l=1"))
[2] ((dim=2|id=595|"Qubit,Site,n=2"), (dim=4|id=434|"Link,l=2"), (dim=2|id=729|"Link,l=1"))
[3] ((dim=2|id=611|"Qubit,Site,n=3"), (dim=8|id=118|"Link,l=3"), (dim=4|id=434|"Link,l=2"))
[4] ((dim=2|id=710|"Qubit,Site,n=4"), (dim=4|id=223|"Link,l=4"), (dim=8|id=118|"Link,l=3"))
[5] ((dim=2|id=840|"Qubit,Site,n=5"), (dim=2|id=920|"Link,l=5"), (dim=4|id=223|"Link,l=4"))
[6] ((dim=2|id=890|"Qubit,Site,n=6"), (dim=2|id=920|"Link,l=5"))
, Dict(:κEA => [2.0, 2.0, 1.00000256, 1.00000256, 1.0000000000065536, 1.0000000000065536], :SR => [1.0, 1.0, 0.9999999999999994, 0.9999999999999998, 1.0000000000000002, 0.9999999999999997], :maxlinkdim => [4.0, 4.0, 8.0, 8.0, 8.0, 8.0], :Ic => [1.0, 1.0, 0.11086058934956577, 0.11086058934956533, 0.00460924147176045, 0.0046092414717571195], :κ2 => [2.0, 2.0, 1.9999999999999996, 2.0, 2.0000000000000018, 2.0000000000000018]))

In [9]:
λ = 0.5
δ = 0.7
q = 0.1
L = 4
T = 2L+2

_, data = circuit(L, T, λ, δ, q; observables=[:Ic, :SR, :κEA, :κ2, :maxlinkdim])

(MPS
[1] ((dim=2|id=809|"Qubit,Site,n=1"), (dim=2|id=382|"Link,l=1"))
[2] ((dim=2|id=730|"Qubit,Site,n=2"), (dim=4|id=835|"Link,l=2"), (dim=2|id=382|"Link,l=1"))
[3] ((dim=2|id=235|"Qubit,Site,n=3"), (dim=8|id=321|"Link,l=3"), (dim=4|id=835|"Link,l=2"))
[4] ((dim=2|id=2|"Qubit,Site,n=4"), (dim=16|id=373|"Link,l=4"), (dim=8|id=321|"Link,l=3"))
[5] ((dim=2|id=741|"Qubit,Site,n=5"), (dim=22|id=24|"Link,l=5"), (dim=16|id=373|"Link,l=4"))
[6] ((dim=2|id=697|"Qubit,Site,n=6"), (dim=16|id=582|"Link,l=6"), (dim=22|id=24|"Link,l=5"))
[7] ((dim=2|id=383|"Qubit,Site,n=7"), (dim=8|id=402|"Link,l=7"), (dim=16|id=582|"Link,l=6"))
[8] ((dim=2|id=702|"Qubit,Site,n=8"), (dim=4|id=199|"Link,l=8"), (dim=8|id=402|"Link,l=7"))
[9] ((dim=2|id=206|"Qubit,Site,n=9"), (dim=2|id=497|"Link,l=9"), (dim=4|id=199|"Link,l=8"))
[10] ((dim=2|id=963|"Qubit,Site,n=10"), (dim=2|id=497|"Link,l=9"))
, Dict(:κEA => [4.0, 4.0, 1.0701984227566563, 1.2422790001974904, 1.0141872487238452, 1.7011987489669262, 1.0445229298319427,

In [17]:
to_matrix(ρ) ≈ to_matrix(ρ2)

true

In [17]:
# data = Dict{Tuple{Int, Int, Float64, Float64, Float64},Tuple{Int,Dict}}()

In [ ]:
# observables = Symbol[:κEA, :κ2, :Ic, :SR, :maxlinkdim]
# samples = 250
# δ = 0.7

# # observables = Symbol[:maxlinkdim]

# for q in [0.1]
#     for λ in 0.0:0.1:1.0
#         for L in 4:4:12
#             T = 2L
#             data[(L,T,λ,δ,q)] = (samples, sample(L, T, λ, δ, q, samples; observables=observables))
#         end
#     end
# end

In [ ]:
# jldsave("25_07_22.jld2"; data)

In [ ]:
data

Dict{Tuple{Int64, Int64, Float64, Float64, Float64}, Tuple{Int64, Dict}} with 33 entries:
  (8, 16, 0.6, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 1.06778, 1.57057, 1.00…
  (12, 24, 0.0, 0.7, 0.1) => (250, Dict(:κEA=>[12.0, 12.0, 2.84549, 9.76532, 2.…
  (8, 16, 0.1, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 2.08585, 5.74068, 1.73…
  (12, 24, 0.3, 0.7, 0.1) => (250, Dict(:κEA=>[12.0, 12.0, 1.91091, 6.15861, 1.…
  (4, 8, 0.4, 0.7, 0.1)   => (250, Dict(:κEA=>[4.0, 4.0, 1.1432, 2.13562, 1.056…
  (8, 16, 1.0, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 1.00022, 1.00022, 1.00…
  (4, 8, 0.6, 0.7, 0.1)   => (250, Dict(:κEA=>[4.0, 4.0, 1.02905, 1.37289, 1.00…
  (4, 8, 0.1, 0.7, 0.1)   => (250, Dict(:κEA=>[4.0, 4.0, 1.46536, 3.11431, 1.32…
  (8, 16, 0.2, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 1.85823, 5.36032, 1.53…
  (8, 16, 0.7, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 1.02335, 1.20661, 1.00…
  (8, 16, 0.8, 0.7, 0.1)  => (250, Dict(:κEA=>[8.0, 8.0, 1.00653, 1.04817, 1.00…
  (12, 24, 0.9, 0.7